# Training Regression Models

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/ageron/handson-ml2/blob/master/04_training_linear_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
  </td>
  <td>
    <a target="_blank" href="https://kaggle.com/kernels/welcome?src=https://github.com/ageron/handson-ml2/blob/master/04_training_linear_models.ipynb"><img src="https://kaggle.com/static/images/open-in-kaggle.svg" /></a>
  </td>
</table>

# Setup

Following is a similar setup that we did for the end-to-end ML project.

Let's import a few common modules, ensure MatplotLib plots figures inline and prepare a function to save the figures. We also check that Python 3.5 or later is installed (although Python 2.x may work, it is deprecated so we strongly recommend you use Python 3 instead), as well as Scikit-Learn ≥0.20.



In [ ]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "training_linear_models"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

# Linear Regression

## The Normal Equation

We assume that
\begin{equation}
    \hat{y} = \theta_0 + \theta_1 x_1 + \theta_2 x_2 + \ldots + \theta_n x_n \;(\;+ \;\epsilon),
\end{equation}
where,

$n$ is the number of features,<br>
$\hat{y}$ is the predicted value,<br>
$x_i$ is the $i^{th}$ feature,<br>
$\theta_0$ is a paramter representing the **intercept** or the **bias**,<br>
$\theta_i$ is the weight or coefficient of $i^{th}$ feature.<br>

In vectorized form, we will write it as:

\begin{equation}
\hat{y} = \boldsymbol{h_{\theta}(x)} = \boldsymbol{\theta}^\intercal \mathbf{x},
\end{equation}

where $\boldsymbol{h_{\theta}(x)}$ is the hypothesis functions over the parameters $\boldsymbol{\theta}$.

**Use an existing data set or create an example data set.**

In [ ]:
import numpy as np

X = 2 * np.random.rand(100, 1)
y = 4 + 3 * X + np.random.randn(100, 1)

In [ ]:
np.random.rand(100, 1)

In [ ]:
plt.plot(X, y, "b.")
plt.xlabel("$x_1$", fontsize=18)
plt.ylabel("$y$", rotation=0, fontsize=18)
plt.axis([0, 2, 0, 15])
save_fig("generated_data_plot")
plt.show()

**<font color='red'> Q: How to find the regression parameters? </font>**</br>


- performance measure/loss function?
- this function may have a special name in the literature
- how to optimize?


**Calculating the regression parameters using a numpy function.**

In [ ]:
# Adding a column of ones as the first column to the X matrix (why?)
X_b = np.c_[np.ones((100, 1)), X]  # concatenates a column (x0 = 1 for each instance) in the beginning
theta_best = np.linalg.inv(X_b.T.dot(X_b)).dot(X_b.T).dot(y)

In [ ]:
theta_best

**<font color='red'> Q: How did we get the closed form solution for theta_best? </font>**</br>
**<font color='red'> Q: Will the inverse always exist? </font>**<br>
**<font color='red'> Q: What is the computational complexity of finding the inverse? </font>**

In [ ]:
theta_best

**<font color='red'> Q: What would be ideal values of the above parameters? </font>**

**Testing the model on a couple of data points.**

In [ ]:
X_new = np.array([[0], [2]])
X_new_b = np.c_[np.ones((2, 1)), X_new]  # add x0 = 1 to each instance
y_predict = X_new_b.dot(theta_best)

print(X_new)
print(X_new_b)
print(y_predict)

In [ ]:
plt.plot(X_new, y_predict, "r-")
plt.plot(X, y, "b.")
plt.axis([0, 2, 0, 15])
plt.show()

Plot with a legend and axis labels.

In [ ]:
plt.plot(X_new, y_predict, "r-", linewidth=2, label="Predictions")
plt.plot(X, y, "b.")
plt.xlabel("$x_1$", fontsize=18)
plt.ylabel("$y$", rotation=0, fontsize=18)
plt.legend(loc="upper left", fontsize=14)
plt.axis([0, 2, 0, 15])
save_fig("linear_model_predictions_plot")
plt.show()

**Using the linear regression module from Scikit-learn.**
- It uses Singular Value Decomposition (SVD): decompose the training set matrix X as a multiplication of three matrices, U Σ $V^\intercal$, using eigenvectors and pseudoinverse.
- Computational complexity is better, and can work in presence of singularity

In [ ]:
from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression()
lin_reg.fit(X, y)
lin_reg.intercept_, lin_reg.coef_

In [ ]:
lin_reg.predict(X_new)

The `LinearRegression` class is based on the `scipy.linalg.lstsq()` function (the name stands for "least squares"), which you could call directly:

In [ ]:
theta_best_svd, residuals, rank, s = np.linalg.lstsq(X_b, y, rcond=1e-6)
theta_best_svd

The following function presents a computationally better alternative to computing the full inverse. It computes $\mathbf{X}^+\mathbf{y}$, where $\mathbf{X}^{+}$ is the _pseudoinverse_ of $\mathbf{X}$ (specifically the Moore-Penrose inverse). You can use `np.linalg.pinv()` to compute the pseudoinverse directly.

In [ ]:
np.linalg.pinv(X_b).dot(y)

#### Find the MSE/RMSE

In [ ]:
from sklearn.metrics import mean_squared_error

y_predictions = lin_reg.predict(X)
lin_mse = mean_squared_error(y, y_predictions)
lin_rmse = np.sqrt(lin_mse)
lin_rmse

**<font color='red'> Q: Can we implement the same using numpy or basic Python functions? </font>**<br>

**<font color='red'> Q: What is the computational complexity of prediction? </font>**<br>
- Both approaches (solving the Normal Equation and the SVD approach) get slow when the number of features
grows large
- Both are linear in terms of the number of instances in the training set (they are O(m)), so can handle large training sets efficiently (provided they can fit in memory!)

# Approaches for Big Data Sets

# Gradient Descent

<!-- ![title](images/gradientDescent.png) -->
<!-- <div>
<img src="attachment:images/gradientDescent.png" width="100"/>
</div> -->
<div> <img src="images/gradientDescent.png" alt="Drawing" style="width: 500px;"/></div>
                                             Source: Géron, Aurélien (2019)

**<font color='red'> Q: What is the significance of the learning step? </font>**<br>
- what if it is too small?
- what if it is too high?

**<font color='red'> Q: What is the significance of the starting point? </font>**<br>

**<font color='red'> Q: Other variants? </font>**<br>

<div> <img src="images/nonconvexGradientDescent.png" alt="Drawing" style="width: 500px;"/></div>
                                             Source: Géron, Aurélien (2019)

**<font color='red'> Q: What kind of function is the least square minimization? </font>**<br>
**<font color='red'> Q: Do algorithms vary based on the type of functions? </font>**<br>
- Linear
- Convex Quadratic
- Nonconvex Quadratic
- General (nonconvex) nonlinear
- Presence of Integer Constrained Variables
- Blackbox/simulation-based functions
- etc.

Gradient Descent is proven to approach arbitrarily close to a minimum (asymptotically, under certain conditions e.g. "sufficient" learning rate)

#### For quadratic functions, gradient descent converges faster.
**<font color='red'> Q: Is the function on the right quadratic? </font>**<br>
**<font color='red'> Q: What about the convergence rate for such functions? </font>**<br>

<div> <img src="images/effectOfScaling.png" alt="Drawing" style="width: 600px;"/></div>
                                             Source: Géron, Aurélien (2019)

## Batch Gradient Descent

A gradient descent step:
\begin{equation}
\boldsymbol{\theta^{i+1} = \theta^{i}} − \eta \nabla_\theta L(\boldsymbol{\theta})
\end{equation}

In [ ]:
eta = 0.1  # learning rate
n_iterations = 1000
m = 100

theta = np.random.randn(2,1)  # random initialization

for iteration in range(n_iterations):
    gradients = 2/m * X_b.T.dot(X_b.dot(theta) - y)
    theta = theta - eta * gradients

In [ ]:
theta

In [ ]:
X_new_b.dot(theta)

In [ ]:
theta_path_bgd = []

def plot_gradient_descent(theta, eta, theta_path=None):
    m = len(X_b)
    plt.plot(X, y, "b.")
    n_iterations = 1000
    for iteration in range(n_iterations):
        if iteration < 10:
            y_predict = X_new_b.dot(theta)
            style = "b-" if iteration > 0 else "r--"
            plt.plot(X_new, y_predict, style)
        gradients = 2/m * X_b.T.dot(X_b.dot(theta) - y)
        theta = theta - eta * gradients
        if theta_path is not None:
            theta_path.append(theta)
    plt.xlabel("$x_1$", fontsize=18)
    plt.axis([0, 2, 0, 15])
    plt.title(r"$\eta = {}$".format(eta), fontsize=16)

In [ ]:
np.random.seed(42)
theta = np.random.randn(2,1)  # random initialization

plt.figure(figsize=(10,4))
plt.subplot(131); plot_gradient_descent(theta, eta=0.02)
plt.ylabel("$y$", rotation=0, fontsize=18)
plt.subplot(132); plot_gradient_descent(theta, eta=0.1, theta_path=theta_path_bgd)
plt.subplot(133); plot_gradient_descent(theta, eta=0.5)

save_fig("gradient_descent_plot")
plt.show()

**<font color='red'> Q: What do you think is a major computational issue with batch gradient descent? </font>**

## Stochastic Gradient Descent

- can be implemented as an out-of-core algorithm
- picks a random small batch (or instance) in the training set at every step and computes
the gradients based only on that
- can escape local minima!

<div> <img src="images/stochasticGradientDescent.png" alt="Drawing" style="width: 500px;"/></div>
                                             Source: Géron, Aurélien (2019)

In [ ]:
theta_path_sgd = []
m = len(X_b)
np.random.seed(42)

In [ ]:
n_epochs = 50
t0, t1 = 5, 50  # learning schedule hyperparameters

def learning_schedule(t):
    return t0 / (t + t1)

theta = np.random.randn(2,1)  # random initialization

for epoch in range(n_epochs):
    for i in range(m):
        if epoch == 0 and i < 20:                    # not shown in the book
            y_predict = X_new_b.dot(theta)           # not shown
            style = "b-" if i > 0 else "r--"         # not shown
            plt.plot(X_new, y_predict, style)        # not shown
        random_index = np.random.randint(m)
        xi = X_b[random_index:random_index+1]
        yi = y[random_index:random_index+1]
        gradients = 2 * xi.T.dot(xi.dot(theta) - yi)
        eta = learning_schedule(epoch * m + i)
        theta = theta - eta * gradients
        theta_path_sgd.append(theta)                 # not shown

plt.plot(X, y, "b.")                                 # not shown
plt.xlabel("$x_1$", fontsize=18)                     # not shown
plt.ylabel("$y$", rotation=0, fontsize=18)           # not shown
plt.axis([0, 2, 0, 15])                              # not shown
save_fig("sgd_plot")                                 # not shown
plt.show()                                           # not shown

In [ ]:
theta

In [ ]:
from sklearn.linear_model import SGDRegressor

sgd_reg = SGDRegressor(max_iter=1000, tol=1e-3, penalty=None, eta0=0.1, random_state=42)
sgd_reg.fit(X, y.ravel())

In [ ]:
sgd_reg.intercept_, sgd_reg.coef_

**<font color='red'> Q: Can we do better than taking just one data point in each iteration? </font>**

## Mini-batch gradient descent

In [ ]:
theta_path_mgd = []

n_iterations = 50
minibatch_size = 20

np.random.seed(42)
theta = np.random.randn(2,1)  # random initialization

t0, t1 = 200, 1000
def learning_schedule(t):
    return t0 / (t + t1)

t = 0
for epoch in range(n_iterations):
    shuffled_indices = np.random.permutation(m)
    X_b_shuffled = X_b[shuffled_indices]
    y_shuffled = y[shuffled_indices]
    for i in range(0, m, minibatch_size):
        t += 1
        xi = X_b_shuffled[i:i+minibatch_size]
        yi = y_shuffled[i:i+minibatch_size]
        gradients = 2/minibatch_size * xi.T.dot(xi.dot(theta) - yi)
        eta = learning_schedule(t)
        theta = theta - eta * gradients
        theta_path_mgd.append(theta)

In [ ]:
theta

In [ ]:
theta_path_bgd = np.array(theta_path_bgd)
theta_path_sgd = np.array(theta_path_sgd)
theta_path_mgd = np.array(theta_path_mgd)

In [ ]:
plt.figure(figsize=(7,4))
plt.plot(theta_path_sgd[:, 0], theta_path_sgd[:, 1], "r-s", linewidth=1, label="Stochastic")
plt.plot(theta_path_mgd[:, 0], theta_path_mgd[:, 1], "g-+", linewidth=2, label="Mini-batch")
plt.plot(theta_path_bgd[:, 0], theta_path_bgd[:, 1], "b-o", linewidth=3, label="Batch")
plt.legend(loc="upper left", fontsize=16)
plt.xlabel(r"$\theta_0$", fontsize=20)
plt.ylabel(r"$\theta_1$   ", fontsize=20, rotation=0)
plt.axis([2.5, 4.5, 2.3, 3.9])
save_fig("gradient_descent_paths_plot")
plt.show()

**<font color='red'> What if the data is not inherently linear? </font>**

# Polynomial Regression

In [ ]:
import numpy as np
import numpy.random as rnd

np.random.seed(42)

In [ ]:
m = 100
X = 6 * np.random.rand(m, 1) - 3
y = 0.5 * X**2 + X + 2 + np.random.randn(m, 1)

In [ ]:
plt.plot(X, y, "b.")
plt.xlabel("$x_1$", fontsize=18)
plt.ylabel("$y$", rotation=0, fontsize=18)
plt.axis([-3, 3, 0, 10])
save_fig("quadratic_data_plot")
plt.show()

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
poly_features = PolynomialFeatures(degree=2, include_bias=False)
X_poly = poly_features.fit_transform(X)
X[0]

PolynomialFeatures includes all combinations of features up to the given degree to the polynomial regression.

In [ ]:
X_poly[0]

#### Let's try the regression now.

In [ ]:
lin_reg = LinearRegression()
lin_reg.fit(X_poly, y)
lin_reg.intercept_, lin_reg.coef_

In [ ]:
X_new=np.linspace(-3, 3, 100).reshape(100, 1)
X_new_poly = poly_features.transform(X_new)
y_new = lin_reg.predict(X_new_poly)
plt.plot(X, y, "b.")
plt.plot(X_new, y_new, "r-", linewidth=2, label="Predictions")
plt.xlabel("$x_1$", fontsize=18)
plt.ylabel("$y$", rotation=0, fontsize=18)
plt.legend(loc="upper left", fontsize=14)
plt.axis([-3, 3, 0, 10])
save_fig("quadratic_predictions_plot")
plt.show()

#### Let's try a few polynomials on our data.

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

for style, width, degree in (("g-", 1, 300), ("b--", 2, 2), ("r-+", 2, 1)):
    polybig_features = PolynomialFeatures(degree=degree, include_bias=False)
    std_scaler = StandardScaler()
    lin_reg = LinearRegression()
    polynomial_regression = Pipeline([
            ("poly_features", polybig_features),
            ("std_scaler", std_scaler),
            ("lin_reg", lin_reg),
        ])
    polynomial_regression.fit(X, y)
    y_newbig = polynomial_regression.predict(X_new)
    plt.plot(X_new, y_newbig, style, label=str(degree), linewidth=width)

plt.plot(X, y, "b.", linewidth=3)
plt.legend(loc="upper left")
plt.xlabel("$x_1$", fontsize=18)
plt.ylabel("$y$", rotation=0, fontsize=18)
plt.axis([-3, 3, 0, 10])
save_fig("high_degree_polynomials_plot")
plt.show()

**<font color='red'> Which model would generalize the best in this case? </font>**<br>

**<font color='red'> How can you decide how complex your model should be? </font>**

**<font color='red'> How can you tell that your model is overfitting or underfitting the data? </font>**

    


## Detecting Underfitting and Overfitting

### (K-fold) Cross Validation
- If a model performs well on the training data but generalizes with respect to the cross-validation metrics, then the model is ______fitting?

- If it performs poorly on both, then it is ______fitting?

Another way is to use learning curves.

### Learning Curves

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

def plot_learning_curves(model, X, y):
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)
    train_errors, val_errors = [], []
    for m in range(1, len(X_train)):
        model.fit(X_train[:m], y_train[:m])
        y_train_predict = model.predict(X_train[:m])
        y_val_predict = model.predict(X_val)
        train_errors.append(mean_squared_error(y_train[:m], y_train_predict))
        val_errors.append(mean_squared_error(y_val, y_val_predict))
    plt.plot(np.sqrt(train_errors), "r-+", linewidth=2, label="train")
    plt.plot(np.sqrt(val_errors), "b-", linewidth=3, label="val")
    plt.axis([1, 80, 0, 4])
    plt.legend(loc="upper right")
    plt.xlabel("Size of the training set", fontsize=14)
    plt.ylabel("$RMSE$", rotation=90, fontsize=14)

On Linear Regression

In [ ]:
lin_reg = LinearRegression()
plot_learning_curves(lin_reg, X, y)

Performance on the 
- training set: the model fits well when there are few instances but can not keep up. 
- validation set: does not generalize well initially, but then learns

**<font color='red'> The error stabilizes, and at about similar values for both the curves. Why? </font>**<br>

**<font color='red'> Both curves have reach a plateau; they are close to each other and fairly high. What does that indicate? </font>**<br>

On Polynomial Regression

In [ ]:
from sklearn.pipeline import Pipeline
polynomial_regression = Pipeline([
("poly_features", PolynomialFeatures(degree=10, include_bias=False)),
("lin_reg", LinearRegression()),
])
plot_learning_curves(polynomial_regression, X, y)

- The error on the training set is lower that the Linear Regression model.
- If there is gap between the curves, it indicates overfitting:
    - if the curves get closer with more data, the gap could go away
    - regularize the model, or choose a simpler one and try again

## The bias/variance trade-off
A model’s generalization error results from the following different errors:
- Bias: due to wrong assumptions e.g. choosing linear model on a quadratic data.
    - A high-bias model will most likely underfit the training data
- Variance: excessive sensitivity to small variations
    - A relatively complex model with many degrees of freedom (such as a high-degree polynomial model) will most likely have high variance and overfit the training data
- Irreducible error: due to noisiness of the data itself; fix the data and the data sources

Increasing the complexity of model results in: high/low(?) bias and high/low(?) variance.

## Regularization of Models
To remedy Overfitting: "constrain" the model parameters (or reducing the degrees of freedom) to make it difficult to overfit.

For example, a linear regression model can be regularized by constraining the $\theta$s

### Ridge Regression

Linear Regression with regularization (also called Tikhonov regularization):
- add the term $\alpha \sum\limits_{i=1}^n \theta_i^2$ to the loss function $L(\theta)$ or $MSE(\theta)$
- Ridge Regression loss/cost function: $R(\theta) := MSE(\theta) + \frac{\alpha}{2} \sum\limits_{i=1}^n \theta_i^2$
- this forces the optimization algorithms to minimize the model weights as well (in addition to minimizing the MSE)
- the regularization term is added to the cost function only while training

Note: Almost all regularized models are sensitive to scaling.

Using Scikit-Learn

In [ ]:
from sklearn.linear_model import Ridge
ridge_reg = Ridge(alpha=1, solver="cholesky")
ridge_reg.fit(X, y)
ridge_reg.predict([[1.5]])

Using Stochastic Gradient Descent

In [ ]:
sgd_reg = SGDRegressor(penalty="l2")
sgd_reg.fit(X, y.ravel())
sgd_reg.predict([[1.5]])

**<font color='red'> Is $\alpha$ a model parameter or a hyperparameter? </font>**<br>

**<font color='red'> What is the effect of having a value of $\alpha$ that is: </font>**<br>
**<font color='red'> (i) very small? </font>**<br>
**<font color='red'> (ii) very large? </font>**


<div> <img src="images/ridgeRegression.png" alt="Drawing" style="width: 600px;"/></div>
                                             Source: Géron, Aurélien (2019)

Different "norms" in the regularization term yield different versions of Regularized regressions. For example,  Lasso Regression uses 1-norm: $\alpha \sum\limits_{i=1}^n \theta_i$.